In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import splitfolders
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import keras_tuner as kt
import scipy
import os
import numpy as np
from modelfunc import save_model

In [2]:
np.random.seed(123)

In [3]:
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.get_logger().setLevel('ERROR')

#### Image Parameters

In [5]:
batch_size = 32
img_height = 256
img_width = 256
img_channels = 3

#### Data

In [6]:
train_split_test_dir='../0. Project Data/Reshaped zero-padded and split data'
train_ds = image_dataset_from_directory(
    train_split_test_dir + '/train',
    image_size=(img_width, img_height),
    batch_size=batch_size)
val_ds = image_dataset_from_directory(
    train_split_test_dir + '/val',
    image_size=(img_width, img_height),
    batch_size=batch_size)
test_ds = image_dataset_from_directory(
    train_split_test_dir + '/test',
    image_size=(img_width, img_height),
    batch_size=batch_size)

Found 2678 files belonging to 4 classes.
Found 667 files belonging to 4 classes.
Found 843 files belonging to 4 classes.


In [7]:
def build_model(hp):
    conv_base = keras.applications.vgg16.VGG16(
        weights='imagenet',
        include_top=False)
    conv_base.trainable = False
    
    data_augmentation = keras.Sequential([layers.RandomFlip('horizontal'), 
                                          layers.RandomRotation(0.1), 
                                          layers.RandomZoom(0.2),])
    
    hp_units = hp.Int('units', min_value=128, max_value=512, step=32)
    hp_dropout = hp.Float('dropout', min_value=.1, max_value=.5, step=.2)
    
    inputs = keras.Input(shape=(img_height, img_width, img_channels))
    x = data_augmentation(inputs)
    x = keras.applications.vgg16.preprocess_input(x)
    x = conv_base(x)
    x = layers.Flatten()(x)
    x = layers.Dense(hp_units)(x)
    x = layers.Dropout(hp_dropout)(x)
    outputs = layers.Dense(4, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model 

In [8]:
tuner=kt.BayesianOptimization(build_model,
                             objective='val_accuracy',
                             max_trials = 100,
                             executions_per_trial=1,
                             directory='kt_logs',
                             overwrite=False); # if changed tuner, set to True

In [9]:
tuner.search_space_summary(extended=True)

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.2, 'sampling': 'linear'}


In [10]:
callbacks = [
 keras.callbacks.EarlyStopping(monitor="val_loss", patience=7),
]

based off previous transfer learning model over fitting very early, we only do 20 epochs

In [11]:
tuner.search(train_ds,
             batch_size=batch_size,
             epochs=20,
             validation_data=val_ds,
             callbacks=callbacks,
             verbose=2,
)

In [12]:
best_hps=tuner.get_best_hyperparameters(num_trials=5)

In [13]:
print(f'''
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is\n
{best_hps[0].get('units')}\n
and the optimal dropout rate for the dropout layer is \n
{best_hps[0].get('dropout')}.
''')


The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is

128

and the optimal dropout rate for the dropout layer is 

0.5.



In [14]:
tuner.results_summary(num_trials=5)

Results summary
Results in kt_logs\untitled_project
Showing 5 best trials
Objective(name="val_accuracy", direction="max")

Trial 063 summary
Hyperparameters:
units: 128
dropout: 0.5
Score: 0.9640179872512817

Trial 048 summary
Hyperparameters:
units: 160
dropout: 0.5
Score: 0.9595202207565308

Trial 075 summary
Hyperparameters:
units: 288
dropout: 0.30000000000000004
Score: 0.9580209851264954

Trial 086 summary
Hyperparameters:
units: 128
dropout: 0.1
Score: 0.95652174949646

Trial 059 summary
Hyperparameters:
units: 128
dropout: 0.5
Score: 0.9550225138664246


In [15]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
    keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', patience=10)
    ]
    history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=callbacks)
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [16]:
combined_train_and_val_ds = tf.data.Dataset.concatenate(train_ds, val_ds).shuffle(buffer_size=100)

In [17]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        combined_train_and_val_ds,
        batch_size=batch_size, epochs=int(best_epoch * 1.25)) #train for more epoch cause more data
    return model

In [18]:
best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(test_ds)
    best_models.append(model)

Epoch 1/100
84/84 [==============================] - 49s 474ms/step - loss: 20.4801 - accuracy: 0.8219 - val_loss: 12.9122 - val_accuracy: 0.8906
Epoch 2/100
84/84 [==============================] - 29s 343ms/step - loss: 10.7732 - accuracy: 0.8951 - val_loss: 9.5392 - val_accuracy: 0.9325
Epoch 3/100
84/84 [==============================] - 29s 341ms/step - loss: 8.4424 - accuracy: 0.9145 - val_loss: 8.5853 - val_accuracy: 0.9310
Epoch 4/100
84/84 [==============================] - 29s 341ms/step - loss: 6.0857 - accuracy: 0.9276 - val_loss: 6.6079 - val_accuracy: 0.9325
Epoch 5/100
84/84 [==============================] - 29s 342ms/step - loss: 5.3276 - accuracy: 0.9361 - val_loss: 6.6721 - val_accuracy: 0.9355
Epoch 6/100
84/84 [==============================] - 29s 342ms/step - loss: 4.1245 - accuracy: 0.9451 - val_loss: 7.6243 - val_accuracy: 0.9220
Epoch 7/100
84/84 [==============================] - 29s 341ms/step - loss: 4.5122 - accuracy: 0.9451 - val_loss: 6.1595 - val_accura

84/84 [==============================] - 29s 344ms/step - loss: 1.3579 - accuracy: 0.9701 - val_loss: 2.8493 - val_accuracy: 0.9535
Epoch 23/100
84/84 [==============================] - 29s 343ms/step - loss: 0.9229 - accuracy: 0.9701 - val_loss: 3.4348 - val_accuracy: 0.9445
Epoch 24/100
84/84 [==============================] - 29s 341ms/step - loss: 1.5348 - accuracy: 0.9656 - val_loss: 3.5308 - val_accuracy: 0.9505
Epoch 25/100
84/84 [==============================] - 29s 341ms/step - loss: 1.2706 - accuracy: 0.9656 - val_loss: 3.0598 - val_accuracy: 0.9565
Epoch 26/100
84/84 [==============================] - 29s 341ms/step - loss: 1.2312 - accuracy: 0.9671 - val_loss: 4.0714 - val_accuracy: 0.9430
Best epoch: 16
Epoch 1/20
105/105 [==============================] - 34s 302ms/step - loss: 16.7214 - accuracy: 0.8410
Epoch 2/20
105/105 [==============================] - 32s 300ms/step - loss: 9.2945 - accuracy: 0.9052
Epoch 3/20
105/105 [==============================] - 32s 301ms/st

105/105 [==============================] - 44s 408ms/step - loss: 1.4447 - accuracy: 0.9564
Epoch 12/23
105/105 [==============================] - 41s 383ms/step - loss: 0.9123 - accuracy: 0.9650
Epoch 13/23
105/105 [==============================] - 42s 394ms/step - loss: 1.5248 - accuracy: 0.9593
Epoch 14/23
105/105 [==============================] - 43s 401ms/step - loss: 1.1787 - accuracy: 0.9623
Epoch 15/23
105/105 [==============================] - 58s 544ms/step - loss: 1.0080 - accuracy: 0.9617
Epoch 16/23
105/105 [==============================] - 44s 415ms/step - loss: 1.3928 - accuracy: 0.9683
Epoch 17/23
105/105 [==============================] - 45s 416ms/step - loss: 1.2791 - accuracy: 0.9650
Epoch 18/23
105/105 [==============================] - 45s 421ms/step - loss: 1.1811 - accuracy: 0.9638
Epoch 19/23
105/105 [==============================] - 45s 418ms/step - loss: 1.5145 - accuracy: 0.9602
Epoch 20/23
105/105 [==============================] - 44s 408ms/step - loss

105/105 [==============================] - 32s 297ms/step - loss: 0.3202 - accuracy: 0.9758
Epoch 25/25
27/27 [==============================] - 4s 143ms/step - loss: 2.5297 - accuracy: 0.9134
Epoch 1/100
84/84 [==============================] - 30s 341ms/step - loss: 20.9524 - accuracy: 0.8234 - val_loss: 9.9616 - val_accuracy: 0.9025
Epoch 2/100
84/84 [==============================] - 28s 330ms/step - loss: 10.8685 - accuracy: 0.8977 - val_loss: 9.7604 - val_accuracy: 0.8906
Epoch 3/100
84/84 [==============================] - 28s 333ms/step - loss: 7.4830 - accuracy: 0.9160 - val_loss: 6.1684 - val_accuracy: 0.9340
Epoch 4/100
84/84 [==============================] - 28s 337ms/step - loss: 6.6466 - accuracy: 0.9309 - val_loss: 6.6594 - val_accuracy: 0.9265
Epoch 5/100
84/84 [==============================] - 28s 333ms/step - loss: 5.1015 - accuracy: 0.9361 - val_loss: 6.9029 - val_accuracy: 0.9205
Epoch 6/100
84/84 [==============================] - 29s 347ms/step - loss: 4.5105 - 

In [19]:
best_models[0].predict(test_ds)

27/27 [==============================] - 4s 146ms/step


array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 8.9166985e-18, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       ...,
       [1.5198408e-08, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [21]:
save_path='../3. Model Selection/saved_models/Transfer Learning Hyperparameter Tuning Zero-padded Data Model/'
for index, hp_mod in enumerate(best_models):
    if not os.path.exists(save_path + str(index + 1) + '/'):
            os.makedirs(save_path + str(index + 1) + '/')
    save_model(hp_mod, save_path + str(index+1) + '/')

Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk


In [ ]:
#below saved files are gitignored

In [22]:
for index, hp_mod in enumerate(best_models):
    if not os.path.exists('./saved_models/my_model' + str(index + 1)):
            os.makedirs('./saved_models/my_model'+ str(index + 1))
    hp_mod.save('./saved_models/my_model'+ str(index+1))

include validation data as part of final training with chosen hyperparameters

find best epoch

retrain on best epoch+validation data extend epoch by a little more cause more data

use test data